#### 2.1.9 _Programming Exercise:_ Generate segwit v1 addresses for a 2-of-2 MuSig aggregate pubkey

In [ ]:
# Generate individual key pairs
privkey1, pubkey1 = generate_key_pair()
privkey2, pubkey2 = generate_key_pair()

# Generate a 2-of-2 aggregate MuSig key
# Method: generate_musig_key(ECPubKey_list)
c_map, agg_pubkey = generate_musig_key([pubkey1, pubkey2])

# Multiply individual keys with challenges
privkey1_c = privkey1 * c_map[pubkey1]
privkey2_c = privkey2 * c_map[pubkey2]
pubkey1_c = pubkey1 * c_map[pubkey1]
pubkey2_c = pubkey2 * c_map[pubkey2]

# Create a segwit v1 address for the MuSig aggregate pubkey
# Method: address = program_to_witness(version_int, program_bytes)
pubkey_data_musig = agg_pubkey.get_bytes()
program_musig = bytes([pubkey_data_musig[0] & 1]) + pubkey_data_musig[1:]
version = 0x01
address_musig = program_to_witness(version, program_musig)
print("2-of-2 musig: ", address_musig)

#### 2.1.11 _Programming Exercise:_ Instantiate a CTransaction object and populate the version, locktime and inputs

In [ ]:
# Construct transaction which spends the musig segwit v1 output
spending_tx = CTransaction()
spending_tx.nVersion = 1
spending_tx.nLockTime = 0
outpoint = COutPoint(tx_musig.sha256, output_index)
spending_tx_in = CTxIn(outpoint = outpoint)
spending_tx.vin = [spending_tx_in]

# Generate new Bitcoin Core wallet address
# Method: addr_string = test.nodes[0].getnewaddress(address_type="bech32")
# Method: decode addr_string with test.nodes[0].getaddressinfo(addr_string)
dest_addr = test.nodes[0].getnewaddress(address_type="bech32")
scriptpubkey = bytes.fromhex(test.nodes[0].getaddressinfo(dest_addr)['scriptPubKey'])
print("Destination address: {}\n".format(dest_addr))

# Determine minimum fee required for mempool acceptance
min_fee = int(test.nodes[0].getmempoolinfo()['mempoolminfee'] * 100000000)

# Complete output which returns funds to Bitcoin Core wallet
# Tip: Construct output with CTxOut(nValue=value_int, scriptPubKey=script_bytes)
# Tip: CTransaction.vout is a list of CTxOut objects.
amount_sat = int(amount_btc * 100000000)
dest_output = CTxOut(nValue=amount_sat-min_fee, scriptPubKey=scriptpubkey)
spending_tx.vout = [dest_output]
print("Spending transaction:\n{}\n".format(spending_tx))

#### 2.1.12 _Programming Exercise:_ Create a valid bip-schnorr signature for the MuSig aggregate pubkey

In [ ]:
# Create sighash for ALL (0x00)
sighash_musig = TaprootSignatureHash(spending_tx, [output], SIGHASH_ALL_TAPROOT, input_index = 0, scriptpath = False)

# Generate individual nonces for participants and an aggregate nonce point
# Remember to negate the individual nonces if necessary
# Method: generate_schnorr_nonce()
# Method: aggregate_schnorr_nonces(nonce_list)
nonce1 = generate_schnorr_nonce()
nonce2 = generate_schnorr_nonce()
R_agg, negated = aggregate_schnorr_nonces([nonce1.get_pubkey(), nonce2.get_pubkey()])

if negated:
    nonce1.negate()
    nonce2.negate()

# Create an aggregate signature
# Method: sign_musig(privkey, nonce, R_agg, agg_pubkey, sighash_musig)
# Method: aggregate_musig_signatures(partial_signature_list, R_agg)
s1 = sign_musig(privkey1_c, nonce1, R_agg, agg_pubkey, sighash_musig)
s2 = sign_musig(privkey2_c, nonce2, R_agg, agg_pubkey, sighash_musig)
sig_agg = aggregate_musig_signatures([s1, s2], R_agg)
print("Aggregate signature is {}\n".format(sig_agg.hex()))

# Construct transaction witness
witness = CScriptWitness()
witness.stack.append(sig_agg)
witness_in = CTxInWitness()
witness_in.scriptWitness = witness
spending_tx.wit.vtxinwit.append(witness_in)

# Serialize Schnorr transaction for broadcast
spending_tx_str = spending_tx.serialize().hex()

# Test mempool acceptance
assert test.nodes[0].testmempoolaccept([spending_tx_str])[0]['allowed']
print("Success!")